# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Import and Read csv file
file="WeatherPy.csv"
weather=pd.read_csv(file)
weather_df=pd.DataFrame(weather)
weather_df.head()

Unnamed: 0          City  Latitude  Longitude  Max Temp  Humidity  \
0           0      Freeport     40.66     -73.58     70.00        83   
1           1        Faanui    -16.48    -151.75     77.76        81   
2           2      Tasiilaq     65.61     -37.64     30.20       100   
3           3      Hermanus    -34.42      19.23     46.99        94   
4           4  Punta Arenas    -53.15     -70.92     32.00        90   

   Cloudiness  Wind_Speed Country        Date  
0           1       10.29      US  1601344686  
1           2       19.33      PF  1601344746  
2          98        1.12      GL  1601344746  
3          38        7.34      ZA  1601344747  
4          20       10.29      CL  1601344730

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Convert humidity into a float to use it as weights for the heat map
hum = weather_df["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(50.0, 50.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hum, dissipating=False, max_intensity=100, point_radius=1)                             

# Add heat layer 
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Filter the data frame by using loc to get the perfect weather
temp_high=weather_df.loc[weather_df['Max Temp']<80,:]
temp_low=temp_high.loc[temp_high['Max Temp']>75,:]
wind_df=temp_low.loc[temp_low['Wind_Speed']<5,:]
new_weather_df=wind_df.loc[wind_df['Cloudiness']==0,:]
new_weather_df

Unnamed: 0           City  Latitude  Longitude  Max Temp  Humidity  \
78           78        Inhapim    -19.55     -42.12     75.20        46   
212         212         Eureka     40.80    -124.16     75.99        69   
267         267  Alta Floresta     -9.88     -56.09     77.68        58   
454         454    Al Bardīyah     31.76      25.09     77.13        66   
481         481           Esna     25.29      32.55     77.00        33   
487         487        Rosetta     31.40      30.42     77.00        88   

     Cloudiness  Wind_Speed Country        Date  
78            0        3.44      BR  1601344774  
212           0        2.59      US  1601344560  
267           0        1.52      BR  1601344861  
454           0        4.81      LY  1601344926  
481           0        2.24      EG  1601344935  
487           0        4.70      EG  1601344938

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Create the new hotel data frame by filtering the specific keys we want using loc
hotel_df=new_weather_df.loc[:,['City','Country','Latitude','Longitude']]
#Add the new key with empty values
hotel_df['Hotel Name']=''
hotel_df

City Country  Latitude  Longitude Hotel Name
78         Inhapim      BR    -19.55     -42.12           
212         Eureka      US     40.80    -124.16           
267  Alta Floresta      BR     -9.88     -56.09           
454    Al Bardīyah      LY     31.76      25.09           
481           Esna      EG     25.29      32.55           
487        Rosetta      EG     31.40      30.42

In [8]:
#Call up the gmaps url to search for the nearby hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#Name the parameters that will be used to search for the hotels
params = {
    "keyword": "hotel",  
    "radius": 5000,
    "type": "hotel",
    "key": g_key,
}

In [9]:
#Iterate through the data frame rows to get the information required to find the hotels
for index, row in hotel_df.iterrows(): 
    lat=row['Latitude']
    lng=row['Longitude']
    city=row['City']
    params['location']=f'{lat},{lng}'
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 78: Inhapim.
Closest hotel in Inhapim is hotel Carreiro em Inhapim/MG.
------------
Retrieving Results for Index 212: Eureka.
Closest hotel in Eureka is Best Western Plus Humboldt Bay Inn.
------------
Retrieving Results for Index 267: Alta Floresta.
Closest hotel in Alta Floresta is Lisboa Palace Hotel.
------------
Retrieving Results for Index 454: Al Bardīyah.
Closest hotel in Al Bardīyah is Hotel Burdy.
------------
Retrieving Results for Index 481: Esna.
Closest hotel in Esna is Sandal Ankh1.
------------
Retrieving Results for Index 487: Rosetta.
Closest hotel in Rosetta is Mecca Hotel.
------------


In [10]:
hotel_df

City Country  Latitude  Longitude  \
78         Inhapim      BR    -19.55     -42.12   
212         Eureka      US     40.80    -124.16   
267  Alta Floresta      BR     -9.88     -56.09   
454    Al Bardīyah      LY     31.76      25.09   
481           Esna      EG     25.29      32.55   
487        Rosetta      EG     31.40      30.42   

                             Hotel Name  
78         hotel Carreiro em Inhapim/MG  
212  Best Western Plus Humboldt Bay Inn  
267                 Lisboa Palace Hotel  
454                         Hotel Burdy  
481                        Sandal Ankh1  
487                         Mecca Hotel

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,
    info_box_content=hotel_info)
#Add layer markers
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))